In [ ]:
import numpy as np
from scipy.sparse import csr_matrix, find
from scipy.spatial import cKDTree
#from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm as tqdm

from datasets.graph import draw_sample
import torch
import torch_geometric
import torch
import os
import os.path as osp


In [ ]:
# import photon clusters
import glob
raw_dir= '/home/sameasy2006/npz_hgcal_50k_k8/HGCNTUP_photon_3to1000_50k/clusters/'
fnamelist = [filepath for filepath in glob.glob(raw_dir+'data_*.pt')]
data_list_pho = []
for i in tqdm(fnamelist):
    idx = torch.load(i)
    #print(idx.y)
    idx.y = np.log(idx.y)
    #idx.w = phoweight
    idx.z = torch.tensor([[0,idx.y]])
    #print(idx.y)
    data_list_pho.append(idx)
    #data_list.append(torch.load(i))
    

#data_list=[x for x in data_list if (x.y<100.)]   
print((data_list_pho[0].y))
print((data_list_pho[0].z))
print(data_list_pho[0].pos)
print(len(data_list_pho))
totalevpho = len(data_list_pho)
#trainev = 

In [ ]:
# import pion clusters
import glob
raw_dir='/home/sameasy2006/npz_hgcal_pion_50k_k8/HGCNTUP_pion_3to1000_50k/clusters/'
fnamelist = [filepath for filepath in glob.glob(raw_dir+'data_*.pt')]
data_list_pi = []

for i in tqdm(fnamelist):
    idx = torch.load(i)
    #print(idx.y)
    idx.y = np.log(idx.y)
    #idx.w = piweight
    idx.z = torch.tensor([[1,idx.y]])
    #print(idx.y)
    data_list_pi.append(idx)
    #data_list.append(torch.load(i))
    
#data_list=[x for x in data_list if (x.y > 800.)]     
print((data_list_pi[0].y))
#print((data_list_pi[0].x))
print((data_list_pi[0].z))
print(data_list_pi[0].pos)
print(len(data_list_pi))
totalevpi = len(data_list_pi)
#trainev = 
#print((data_list_pi[0].batch))

In [ ]:
#del data_list_comb
import random

data_list_comb = data_list_pho + data_list_pi[:totalevpho]
#print(data_list_comb[0])
random.shuffle(data_list_comb)
#print(data_list_comb[0])
totalev = len(data_list_comb)
print("total comb evs:",totalev)
import torch_geometric
ntrainbatch = 50
ntestbatch = 20
trainloader = torch_geometric.data.DataLoader(data_list_comb[:totalev-500], batch_size=ntrainbatch)
testloader = torch_geometric.data.DataLoader(data_list_comb[totalev-500:totalev], batch_size=ntestbatch)
#batch_size = ntrainbatch
epoch_size = len(data_list_comb[:totalev-500])
print("epoch size,batch_size:",epoch_size,ntrainbatch)

In [ ]:
import os
import os.path as osp
import math

import numpy as np
import torch
import gc
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T

from torch.utils.checkpoint import checkpoint
from torch_cluster import knn_graph

from torch_geometric.nn import EdgeConv, NNConv
#from torch_geometric.nn.pool.edge_pool import EdgePooling

from torch_geometric.utils import normalized_cut
from torch_geometric.utils import remove_self_loops
from torch_geometric.utils.undirected import to_undirected
from torch_geometric.nn import (graclus, max_pool, max_pool_x,
                                global_mean_pool, global_max_pool,
                                global_add_pool)

transform = T.Cartesian(cat=False)

def normalized_cut_2d(edge_index, pos):
    row, col = edge_index
    edge_attr = torch.norm(pos[row] - pos[col], p=2, dim=1)
    return normalized_cut(edge_index, edge_attr, num_nodes=pos.size(0))

class DynamicReductionNetwork(nn.Module):
    # This model iteratively contracts nearest neighbour graphs 
    # until there is one output node.
    # The latent space trained to group useful features at each level
    # of aggregration.
    # This allows single quantities to be regressed from complex point counts
    # in a location and orientation invariant way.
    # One encoding layer is used to abstract away the input features.
    def __init__(self, input_dim=5, hidden_dim=64, output_dim=1, k=16, aggr='add',
                 norm=torch.tensor([1./500., 1./500., 1./54., 1/25., 1./1000.])):
 #                norm=torch.tensor([1., 1., 1., 1., 1.])):
        super(DynamicReductionNetwork, self).__init__()

        self.datanorm = nn.Parameter(norm)
        
        self.k = k
        start_width = 2 * hidden_dim
        middle_width = 3 * hidden_dim // 2

        
        self.inputnet =  nn.Sequential(
            nn.Linear(input_dim, hidden_dim*2),            
            nn.ELU(),
            nn.Linear(hidden_dim*2, hidden_dim*2),
            nn.ELU(),
#            nn.Linear(hidden_dim*2, hidden_dim*2),
#            nn.ELU(),
#            nn.Linear(hidden_dim*2, hidden_dim*2),
#            nn.ELU(),
            nn.Linear(hidden_dim*2, hidden_dim),
            nn.ELU(),
        )
        
        
        convnn1 = nn.Sequential(nn.Linear(start_width, middle_width),
                                nn.ELU(),
                                nn.Linear(middle_width, hidden_dim),                                             
                                nn.ELU()
                                )
        convnn2 = nn.Sequential(nn.Linear(start_width, middle_width),
                                nn.ELU(),
                                nn.Linear(middle_width, hidden_dim),                                             
                                nn.ELU()
                                )
        
        convnn3 = nn.Sequential(nn.Linear(start_width, middle_width),
                                nn.ELU(),
                                nn.Linear(middle_width, hidden_dim),                                             
                                nn.ELU()
                                )
                
        self.edgeconv1 = EdgeConv(nn=convnn1, aggr=aggr)
        self.edgeconv2 = EdgeConv(nn=convnn2, aggr=aggr)
        self.edgeconv3 = EdgeConv(nn=convnn3, aggr=aggr)
        
        self.output = nn.Sequential(nn.Linear(hidden_dim, hidden_dim),
                                    nn.ELU(),
                                    #nn.Softplus(),
                                    nn.Linear(hidden_dim, hidden_dim//2),
                                    nn.ELU(),
                                    #nn.Softplus(),
#                                    nn.Linear(hidden_dim//2, hidden_dim//2),#added
 #                                   nn.ELU(),
                                    #nn.Softplus(),
                                    nn.Linear(hidden_dim//2, output_dim)
                                   )
        
        
    def forward(self, data):        
        data.x = self.datanorm * data.x
        data.x = self.inputnet(data.x)
        
        #print(data.batch)
        data.edge_index = to_undirected(knn_graph(data.x, self.k, data.batch, loop=False, flow=self.edgeconv1.flow))
        data.x = self.edgeconv1(data.x, data.edge_index)
        
        weight = normalized_cut_2d(data.edge_index, data.x)
        cluster = graclus(data.edge_index, weight, data.x.size(0))
        data.edge_attr = None
        data = max_pool(cluster, data)
        
        ####
        data.edge_index = to_undirected(knn_graph(data.x, self.k, data.batch, loop=False, flow=self.edgeconv3.flow))
        data.x = self.edgeconv3(data.x, data.edge_index)
        
        weight = normalized_cut_2d(data.edge_index, data.x)
        cluster = graclus(data.edge_index, weight, data.x.size(0))
        data.edge_attr = None
        data = max_pool(cluster, data)
        ####
        ####
        #data.edge_index = to_undirected(knn_graph(data.x, self.k, data.batch, loop=False, flow=self.edgeconv1.flow))
        #data.x = self.edgeconv3(data.x, data.edge_index)
        
        #weight = normalized_cut_2d(data.edge_index, data.x)
        #cluster = graclus(data.edge_index, weight, data.x.size(0))
        #data.edge_attr = None
        #data = max_pool(cluster, data)
        ####
        
        data.edge_index = to_undirected(knn_graph(data.x, self.k, data.batch, loop=False, flow=self.edgeconv2.flow))
        data.x = self.edgeconv2(data.x, data.edge_index)
        
        weight = normalized_cut_2d(data.edge_index, data.x)
        cluster = graclus(data.edge_index, weight, data.x.size(0))
        x, batch = max_pool_x(cluster, data.x, data.batch)

        x = global_max_pool(x, batch)
#        print(self.output(x))
        return self.output(x).squeeze(-1)


In [ ]:
from torch.optim import Optimizer
from torch.optim.lr_scheduler import _LRScheduler
import math
import torch
import sys

class ReduceMaxLROnRestart:
    def __init__(self, ratio=0.75):
        self.ratio = ratio
        
        def __call__(self, eta_min, eta_max):
            return eta_min, eta_max * self.ratio
        
        
class ExpReduceMaxLROnIteration:
    def __init__(self, gamma=1):
        self.gamma = gamma
        
    def __call__(self, eta_min, eta_max, iterations):
        return eta_min, eta_max * self.gamma ** iterations


class CosinePolicy:
    def __call__(self, t_cur, restart_period):
        return 0.5 * (1. + math.cos(math.pi *
                                    (t_cur / restart_period)))
    
    
class ArccosinePolicy:
    def __call__(self, t_cur, restart_period):
        return (math.acos(max(-1, min(1, 2 * t_cur
                                      / restart_period - 1))) / math.pi)
    
    
class TriangularPolicy:
    def __init__(self, triangular_step=0.5):
        self.triangular_step = triangular_step
        
    def __call__(self, t_cur, restart_period):
        inflection_point = self.triangular_step * restart_period
        point_of_triangle = (t_cur / inflection_point
                             if t_cur < inflection_point
                             else 1.0 - (t_cur - inflection_point)
                             / (restart_period - inflection_point))
        return point_of_triangle
    
    
class CyclicLRWithRestarts(_LRScheduler):
    """Decays learning rate with cosine annealing, normalizes weight decay
    hyperparameter value, implements restarts.
    https://arxiv.org/abs/1711.05101
    Args:
        optimizer (Optimizer): Wrapped optimizer.
        batch_size: minibatch size
        epoch_size: training samples per epoch
        restart_period: epoch count in the first restart period
        t_mult: multiplication factor by which the next restart period will expand/shrink
        policy: ["cosine", "arccosine", "triangular", "triangular2", "exp_range"]
        min_lr: minimum allowed learning rate
        verbose: print a message on every restart
        gamma: exponent used in "exp_range" policy
        eta_on_restart_cb: callback executed on every restart, adjusts max or min lr
        eta_on_iteration_cb: callback executed on every iteration, adjusts max or min lr
        triangular_step: adjusts ratio of increasing/decreasing phases for triangular policy
    Example:
        >>> scheduler = CyclicLRWithRestarts(optimizer, 32, 1024, restart_period=5, t_mult=1.2)
        >>> for epoch in range(100):
        >>>     scheduler.step()
        >>>     train(...)
        >>>         ...
        >>>         optimizer.zero_grad()
        >>>         loss.backward()
        >>>         optimizer.step()
        >>>         scheduler.batch_step()
        >>>     validate(...)
    """
    
    def __init__(self, optimizer, batch_size, epoch_size, restart_period=100,
                 t_mult=2, last_epoch=-1, verbose=False,
                 policy="cosine", policy_fn=None, min_lr=1e-7,
                 eta_on_restart_cb=None, eta_on_iteration_cb=None,
                 gamma=1.0, triangular_step=0.5):
        
        if not isinstance(optimizer, Optimizer):
            raise TypeError('{} is not an Optimizer'.format(
                type(optimizer).__name__))
        
        self.optimizer = optimizer
        
        if last_epoch == -1:
            for group in optimizer.param_groups:
                group.setdefault('initial_lr', group['lr'])
                group.setdefault('minimum_lr', min_lr)
        else:
            for i, group in enumerate(optimizer.param_groups):
                if 'initial_lr' not in group:
                    raise KeyError("param 'initial_lr' is not specified "
                                   "in param_groups[{}] when resuming an"
                                   " optimizer".format(i))
                
        self.base_lrs = [group['initial_lr'] for group
                         in optimizer.param_groups]
        
        self.min_lrs = [group['minimum_lr'] for group
                        in optimizer.param_groups]
        
        self.base_weight_decays = [group['weight_decay'] for group
                                   in optimizer.param_groups]
        
        self.policy = policy
        self.eta_on_restart_cb = eta_on_restart_cb
        self.eta_on_iteration_cb = eta_on_iteration_cb
        if policy_fn is not None:
            self.policy_fn = policy_fn
        elif self.policy == "cosine":
            self.policy_fn = CosinePolicy()
        elif self.policy == "arccosine":
            self.policy_fn = ArccosinePolicy()
        elif self.policy == "triangular":
            self.policy_fn = TriangularPolicy(triangular_step=triangular_step)
        elif self.policy == "triangular2":
            self.policy_fn = TriangularPolicy(triangular_step=triangular_step)
            self.eta_on_restart_cb = ReduceMaxLROnRestart(ratio=0.5)
        elif self.policy == "exp_range":
            self.policy_fn = TriangularPolicy(triangular_step=triangular_step)
            self.eta_on_iteration_cb = ExpReduceMaxLROnIteration(gamma=gamma)
            
        self.last_epoch = last_epoch
        self.batch_size = batch_size
        self.epoch_size = epoch_size
        
        self.iteration = 0
        self.total_iterations = 0
        
        self.t_mult = t_mult
        self.verbose = verbose
        self.restart_period = math.ceil(restart_period)
        self.restarts = 0
        self.t_epoch = -1
        self.epoch = -1
        
        self.eta_min = 0
        self.eta_max = 1
        
        self.end_of_period = False
        self.batch_increments = []
        self._set_batch_increment()
        
    def _on_restart(self):
        if self.eta_on_restart_cb is not None:
            self.eta_min, self.eta_max = self.eta_on_restart_cb(self.eta_min,
                                                                self.eta_max)
            
    def _on_iteration(self):
        if self.eta_on_iteration_cb is not None:
            self.eta_min, self.eta_max = self.eta_on_iteration_cb(self.eta_min,
                                                                  self.eta_max,
                                                                  self.total_iterations)
            
    def get_lr(self, t_cur):
        eta_t = (self.eta_min + (self.eta_max - self.eta_min)
                 * self.policy_fn(t_cur, self.restart_period))
        
        weight_decay_norm_multi = math.sqrt(self.batch_size /
                                            (self.epoch_size *
                                             self.restart_period))
        
        lrs = [min_lr + (base_lr - min_lr) * eta_t for base_lr, min_lr
               in zip(self.base_lrs, self.min_lrs)]
        weight_decays = [base_weight_decay #* eta_t * weight_decay_norm_multi
                         for base_weight_decay in self.base_weight_decays]
        
        if (self.t_epoch + 1) % self.restart_period < self.t_epoch:
            self.end_of_period = True
            
        if self.t_epoch % self.restart_period < self.t_epoch:
            if self.verbose:
                print("Restart {} at epoch {}".format(self.restarts + 1,
                                                      self.last_epoch))
            self.restart_period = math.ceil(self.restart_period * self.t_mult)
            self.restarts += 1
            self.t_epoch = 0
            self._on_restart()
            self.end_of_period = False
            
        return zip(lrs, weight_decays)
        
    def _set_batch_increment(self):
        d, r = divmod(self.epoch_size, self.batch_size)
        batches_in_epoch = d + 2 if r > 0 else d + 1
        self.iteration = 0
        self.batch_increments = torch.linspace(0, 1, batches_in_epoch).tolist()
        
    def step(self):
        self.last_epoch += 1
        self.t_epoch += 1
        self._set_batch_increment()
        self.batch_step()
        
    def batch_step(self):
        try:
            t_cur = self.t_epoch + self.batch_increments[self.iteration]
            self._on_iteration()
            self.iteration += 1
            self.total_iterations += 1
        except (IndexError):
            raise StopIteration("Epoch size and batch size used in the "
                                "training loop and while initializing "
                                "scheduler should be the same.")
        
        for param_group, (lr, weight_decay) in zip(self.optimizer.param_groups,
                                                   self.get_lr(t_cur)):
            param_group['lr'] = lr
            param_group['weight_decay'] = weight_decay


In [ ]:
import os.path as osp

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from tqdm import tqdm_notebook as tqdm
#from tqdm import tqdm as tqdm


#from models.DynamicReductionNetwork import DynamicReductionNetwork

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drn = DynamicReductionNetwork(hidden_dim=50,k=16,output_dim=3)
        
    def forward(self, data):
        logits = self.drn(data)
        return F.softplus(logits)
        #return logits
device = torch.device('cuda')#('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
#optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-3)
#optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-3)
scheduler = CyclicLRWithRestarts(optimizer, ntrainbatch, epoch_size, restart_period=400, t_mult=1.2, policy="cosine")
#criterion = torch.nn.MSELoss()


def resoloss2(output,truth):
    batch_size = output.size()[0]
#    mse = F.mse_loss(output, truth, reduction='mean')
    res = torch.sum((output-truth)**2/truth)/batch_size
    #return (mse + 0.2*res)
    return (res)
    

    
def resoloss(outputa,trutha):
    n_bins = np.random.poisson(40) + 2
    minx = torch.min(trutha).detach().cpu()
    maxx = torch.max(trutha).detach().cpu()
    minx = np.exp(minx)
    maxx = np.exp(maxx)
    
    binss = np.arange(minx,maxx,(maxx-minx)/n_bins)
    binss = np.log(binss)
    loss = 0.
    for i in range(binss.size-1):
        tmpo = outputa[ (trutha > binss[i]) &  (trutha < binss[i+1])  ]
        tmpt = trutha[ (trutha > binss[i]) &  (trutha < binss[i+1])  ]
        #print("tmpo,tmpt:",tmpo,tmpt)
        if (list(tmpo.size())[0] == 0 or list(tmpt.size())[0] == 0):
            continue
        #print(list(tmpo.size())[0],list(tmpt.size())[0])
        tmpom = torch.mean(tmpo)
        tmptm = torch.mean(tmpt)
        #print (tmpom,tmptm)
        #if (tmptm == 0):
        #    continue
        
        #    if ((tmpom - tmptm)**2/tmptm is not np.nan):
        #        print ("(tmpom - tmptm)**2/tmptm",(tmpom - tmptm)**2/tmptm)
        loss += (tmpom - tmptm)**2/tmptm
    
    
    if (loss == 0.):
        print("minx,maxx,binss,n_bins:",minx,maxx,binss,n_bins)    
        print(outputa,trutha)
        for i in range(binss.size-1):
            tmpo = outputa[ (trutha > binss[i]) &  (trutha < binss[i+1])  ]
            tmpt = trutha[ (trutha > binss[i]) &  (trutha < binss[i+1])  ]
            print("tmpo,tmpt:",tmpo,tmpt)
            if (list(tmpo.size())[0] == 0 or list(tmpt.size())[0] == 0):
                continue
            #print(list(tmpo.size())[0],list(tmpt.size())[0])
            tmpom = torch.mean(tmpo)
            tmptm = torch.mean(tmpt)
            print (tmpom,tmptm)
            loss += (tmpom - tmptm)**2/tmptm
            print(loss)
        
    #print("loss: ",loss)
    return loss        
losscat = torch.nn.CrossEntropyLoss()    
def categorical_loss(outputa,trutha,alpha):
    total_loss =  losscat(outputa[:,:2],trutha[:,0].long()) + alpha*resoloss(outputa[:,2],trutha[:,1])
    return total_loss
    

#model.train()
def train(epoch):
    model.train()
    torch.cuda.empty_cache()
    scheduler.step()
    loss = []
    for data in tqdm(trainloader):
            data = data.to(device)        
            optimizer.zero_grad()
            result = model(data)
            #lossc = resoloss(result, data.y, data)
            #print("result:",result)
            #print("truth:",data.z)
            lossc = categorical_loss(result, data.z, 1.0)
#            mse = F.mse_loss(result, data.y, reduction='mean')
#            mse = criterion(result, data.y)
#            print('result, y:',result,data.y)
#            print('crit, orig:',criterion(result, data.y),F.mse_loss(result, data.y, reduction='mean'))
            loss.append(lossc.item()) 
            lossc.backward()
#            print(mse)
            optimizer.step()
            scheduler.batch_step()
    print( 'batches for train:',len(loss)) 
#    print('loss',loss)
    print('train loss:',np.mean(np.array(loss)))
    del loss #memtest
#    print(data)

from scipy.stats import norm
import matplotlib.mlab as mlab
import scipy.stats as scs
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
%matplotlib inline

def gaussian(x,  mean,a, sigma):
    return a * np.exp(-((x - mean)**2 / (2 * sigma**2)))

def evaluate(epoch):
        """"Evaluate the model"""
        model.zero_grad()
        torch.cuda.empty_cache()
        model.eval()
#        loss = []
#        frac = []

        pred = []
        true = []
        loss= []
        
        correct = 0
        predc = []
        truec = []
        for data in tqdm(testloader):
            data = data.to(device)        
            result = model(data)
            #lossc = resoloss(result, data.y)
            lossc = categorical_loss(result, data.z, 1.0)
#            print (result.item(),data.y.item())
#            frac.append((result.item() - data.y.item())/data.y.item())
            loss.append(lossc.item())

            for i in result:
                pred.append(i.detach().cpu()[2])
                predc.append(i.detach().cpu()[0:2].argmax())
            for i in data.z.detach():
                true.append(i.detach().cpu()[1])
                truec.append(i.detach().cpu()[0])
            
            
        #print(predc,truec)
        predc = np.array(predc)
        truec = np.array(truec)
        print("accuracy  :",np.equal(predc,truec).sum()/len(truec))
        
        print('batches for test:', len(loss)) 
        print('test loss:',np.mean(np.array(loss)))
#        fracarr = np.array(frac


        preda = np.array(pred)
        truea = np.array(true)
        #preda = preda[:,2] ### added
        #truea = truea[:,1] ### added
        fracarr = (preda - truea)/truea
        #print(preda,truea,fracarr)
        print('pred - true / true mean:',(np.mean(fracarr)))
        print('pred - true / true std:',(np.std(fracarr)))
        (mu, sigma) = norm.fit(fracarr)
        print('mu,sig:',mu,sigma)
        

        bin_heights, bin_borders, _ = plt.hist(fracarr,range=[-2,2], bins=100, label='histogram')
        bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2
        
        try:
            popt, _ = curve_fit(gaussian, bin_centers, bin_heights, p0=[0., 100., 1.],bounds = ([-np.inf,0,0],[np.inf,np.inf,np.inf]))
            x_interval_for_fit = np.linspace(bin_borders[0], bin_borders[-1], 100)
            plt.plot(x_interval_for_fit, gaussian(x_interval_for_fit, *popt), label='fit')
            plt.legend()


            plt.xlabel('pred - true / true')
            plt.ylabel('counts')
            #plt.yscale("log")
            #plt.title(r'$\mathrm{pred - true / true:}\ \mu=%.3f,\ \sigma=%.3f$' %(mu, sigma))
            plt.title(r'$\mathrm{pred - true / true:}\ \mu=%.3f,\ \sigma=%.3f$' %(popt[0], popt[2]))
            plt.grid(True)

            plt.show()

        except RuntimeError:
            print("Error - curve_fit failed")
            plt.xlabel('pred - true / true')
            plt.ylabel('counts')
            #plt.title(r'$\mathrm{pred - true / true:}\ \mu=%.3f,\ \sigma=%.3f$' %(mu, sigma))
            plt.title('pred - true / true fit failed')
            #plt.yscale("log")
            plt.grid(True)

            plt.show()
            
        return np.mean(np.array(loss))
        del pred,true,loss,preda,truea,fracarr  #memtest

# for train

In [ ]:
from tqdm import tqdm_notebook as tqdm
#checkpoint_dir = '/home/sameasy2006/hgcal_ldrd-gravnet2_wip_trainer_args/ouput_regression_testtesttesttest_dyn2addlayer/'
#checkpoint_dir = '/home/sameasy2006/hgcal_ldrd-gravnet2_wip_trainer_args/ouput_regression_add1ip1dyn_binresoloss/'
checkpoint_dir = '/home/sameasy2006/hgcal_ldrd-gravnet2_wip_trainer_args/ouput_regression_phopi_comb_categ/'

os.makedirs(checkpoint_dir, exist_ok=True)
best_loss = 99999999
for epoch in range(1, 100):
    print ('epoch:',epoch)
    train(epoch)
    loss_epoch = evaluate(epoch)
    checkpoint = {
    'epoch': epoch + 1,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict()
    }
    
    
    
    checkpoint_file = 'model_epoch_%03i.pth.tar' % ( epoch )
    torch.save(checkpoint,
                   os.path.join(checkpoint_dir,checkpoint_file ))
    if loss_epoch < best_loss:
        best_loss = loss_epoch
        print('new best test loss:',best_loss)
        torch.save(checkpoint,
                   os.path.join(checkpoint_dir,'model_checkpoint_best.pth.tar' ))

# for train continue

In [ ]:
model_fname = '/home/sameasy2006/hgcal_ldrd-gravnet2_wip_trainer_args/ouput_regression_phopi_comb_categ/model_cont2_epoch_085.pth.tar'

model=Net().to(device)
#model = Net().to(device)
#optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-3)
model.load_state_dict(torch.load(model_fname)['state_dict'])
#modl.eval()
optimizer.load_state_dict(torch.load(model_fname)['optimizer'])

epochlast = torch.load(model_fname)['epoch']

from tqdm import tqdm_notebook as tqdm
#checkpoint_dir = '/home/sameasy2006/hgcal_ldrd-gravnet2_wip_trainer_args/ouput_regression_testtesttesttest_dyn2addlayer/'
#checkpoint_dir = '/home/sameasy2006/hgcal_ldrd-gravnet2_wip_trainer_args/ouput_regression_add1ip1dyn_binresoloss/'
checkpoint_dir = '/home/sameasy2006/hgcal_ldrd-gravnet2_wip_trainer_args/ouput_regression_phopi_comb_categ/'

os.makedirs(checkpoint_dir, exist_ok=True)
best_loss = 99999999
for epoch in range(int(epochlast), int(epochlast+100)):
    print ('epoch:',epoch)
    train(epoch)
    loss_epoch = evaluate(epoch)
    checkpoint = {
    'epoch': epoch + 1,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict()
    }
    
    
    
    checkpoint_file = 'model_cont3_epoch_%03i.pth.tar' % ( epoch )
    torch.save(checkpoint,
                   os.path.join(checkpoint_dir,checkpoint_file ))
    if loss_epoch < best_loss:
        best_loss = loss_epoch
        print('new best test loss:',best_loss)
        torch.save(checkpoint,
                   os.path.join(checkpoint_dir,'model_cont3_checkpoint_best.pth.tar' ))

# for eval

In [ ]:

#model_fname = '/home/sameasy2006/hgcal_ldrd-gravnet2_wip_trainer_args/ouput_regression_testtesttesttest_dyn2addlayer/model_checkpoint_best.pth.tar'
#model_fname = '/home/sameasy2006/hgcal_ldrd-gravnet2_wip_trainer_args/ouput_regression_add1ip1dyn_binresoloss/model_checkpoint_best.pth.tar'
#model_fname = '/home/sameasy2006/hgcal_ldrd-gravnet2_wip_trainer_args/ouput_regression_2dynoutputact/model_checkpoint_best.pth.tar'
#model_fname = '/home/sameasy2006/hgcal_ldrd-gravnet2_wip_trainer_args/ouput_regression_photon50k_scaleLOGSPLIT/model_checkpoint_best.pth.tar'
model_fname = '/home/sameasy2006/hgcal_ldrd-gravnet2_wip_trainer_args/ouput_regression_phopi_comb_categ/model_cont3_checkpoint_best.pth.tar'


mdl=Net().to(device)
mdl.load_state_dict(torch.load(model_fname)['state_dict'])
mdl.eval()


In [ ]:
from tqdm import tqdm_notebook as tqdm
#mdl.zero_grad()
torch.cuda.empty_cache()
#model.eval()
pred = []
true = []
predc = []
misp = []
mist = []
#testloader2 = torch_geometric.data.DataLoader(data_list[totalev-3000:totalev], batch_size=1)
#testloader2 = torch_geometric.data.DataLoader(data_list, batch_size=50)
#valloader = torch_geometric.data.DataLoader(data_list_pho[totalevpho-10000:totalevpho], batch_size=30)
valloader = torch_geometric.data.DataLoader(data_list_pho, batch_size=60)
  
    
#print(pred)
#print(true)
for data in tqdm(valloader):
    data = data.to(device)        
    result = mdl(data)
    #print(data.y)
    #print(result.i http://127.0.0.1:8005/?token=dcee9ad154badcbcaf8c1643ad620820eec6de751c69d92dtem().detach())
    #pred.append(result.item())
    #true.append(data.y.item())
    for i,j in zip(result,data.z.detach()):
        pred.append(i.detach().cpu()[2])
        predc.append(i.detach().cpu()[0:2].argmax())
        if (i.detach().cpu()[0:2].argmax() == 1):
            misp.append(i.detach().cpu()[2])
            mist.append(j.detach().cpu()[1])
        #pred.append(i.item())
#    print (pred) 
    #for i in data.z.detach():
        true.append(j.detach().cpu()[1])


In [ ]:
print(len(pred))
print(len(true))
print(np.unique(np.array(predc),return_counts=True)[1][0])
print("pho acc, miss:",np.unique(np.array(predc),return_counts=True)[1][0]/len(predc),np.unique(np.array(predc),return_counts=True)[1][1]/len(predc))
#plt.hist(np.exp(misp),bins=50)
plt.scatter(np.exp(mist),np.exp(misp),alpha=0.3)
plt.xlabel("true")
plt.ylabel("pred")
plt.show()

In [ ]:
def plotpred(axs,pred,true,e1,e2):
#    preda = np.array(pred)
#    truea = np.array(true)
    fracarr = (pred - true)/true
#    if e2<250 : 
#        bin_heights, bin_borders, _ = plt.hist(fracarr,range=[-2,2], bins=20, label='histogram')
#    else:
    bin_heights, bin_borders, _ = axs.hist(fracarr,range=[-1.0,1.0], bins=100, label='histogram')

    bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2
    popt, pcov = curve_fit(gaussian, bin_centers, bin_heights, p0=[0., 1000., 0.1],bounds = ([-np.inf,0,0],[np.inf,np.inf,np.inf]))
    perr = np.sqrt(np.diag(pcov))
    #print(popt,perr)

    x_interval_for_fit = np.linspace(bin_borders[0], bin_borders[-1], 100)
    axs.plot(x_interval_for_fit, gaussian(x_interval_for_fit, *popt), label='fit')
    axs.legend()


    axs.set_xlabel('pred - true / true')
    axs.set_ylabel('counts')
    #plt.title(r'$\mathrm{pred - true / true:}\ \mu=%.3f,\ \sigma=%.3f$' %(mu, sigma))
    axs.set_title(r'$\mathrm{pred - true / true:}\ \mu=%.3f,\ \sigma=%.3f,\ E=$%i to %i' %(popt[0], popt[2],e1,e2))
    axs.grid(True)
    
#    plt.show()
    return [popt[0], popt[2],perr[0], perr[2]]

In [ ]:
#bins =  np.linspace(0,1000,21)
#bins =  np.linspace(0,250,6)
bins =  np.linspace(0,200,11)
bins2 = np.linspace(250,1000,16)
bins = np.append(bins,bins2)
print(bins)

pred = np.array(pred)
true = np.array(true)

print('before scaling pred',pred[0])
print('before scaling true',true[0])

pred  = np.exp(pred)
true = np.exp(true)
print('after scaling pred',pred[0])
print('after scaling true',true[0])




seoe = []
seoer = []
me = []
mer = []

fig, axs = plt.subplots(6,5, figsize=(40, 40), facecolor='w', edgecolor='k')
axs = axs.ravel()

for i in tqdm(range (bins.size - 1)):
    preda = pred[(true >bins[i]) & (true <bins[i+1]) ]
    truea = true[(true >bins[i]) & (true <bins[i+1]) ]
    vals = plotpred(axs[i],preda,truea,bins[i],bins[i+1])
    #print (vals)
    #seoe.append(vals[1]/bins[i+1])
    seoe.append(vals[1])
    seoer.append(vals[3])
    me.append(vals[0])
    mer.append(vals[2])
    #print (bins[i],bins[i+1])
seoear = np.array(seoe)
seoerar = np.array(seoer)
mear = np.array(me)
merar = np.array(mer)
#print(seoear,'\n',seoerar)
fig.savefig('outputs.pdf')

In [ ]:
print(pred)
print (true)

In [ ]:
plotbinner = lambda t: t+10 if (t<200) else t+25
vfunc = np.vectorize(plotbinner)
newbin = vfunc(bins[:bins.size-1])
#plt.plot(bins[:5]+50,(seoear)*100,yerr=seoerar)
plt.errorbar(newbin,(seoear), yerr=seoerar, fmt='o', ecolor='orangered',color='steelblue', capsize=2,ls='--');
plt.xlabel('Energy')
plt.ylabel('sigma')
plt.grid(True)

In [ ]:
plt.errorbar(newbin,(mear), yerr=merar, fmt='o', ecolor='orangered',color='steelblue', capsize=2,ls='--');
plt.xlabel('Energy')
plt.ylabel('mean')
plt.grid(True)

In [ ]:
#torch.load(i)
def reso(x,stoch,C):
#    return np.sqrt(stoch**2/x + e_dep**2/x**2 + C**2 )
    return np.sqrt(stoch**2/x  + C**2 )
 



popt, pcov = curve_fit(f=reso,xdata=newbin, ydata=seoear, sigma=seoerar, p0=[1.,1.],bounds = ([0,0],[np.inf,np.inf]))
perr = np.sqrt(np.diag(pcov))
print(popt,perr)

x_interval_for_fit = np.linspace(5,1000, 100)
plt.plot(x_interval_for_fit, reso(x_interval_for_fit, *popt), label='fit')
plt.errorbar(newbin,(seoear), yerr=seoerar, fmt='o', ecolor='orangered',color='steelblue', capsize=2,ls='--');
plt.legend()


plt.xlabel('Energy')
plt.ylabel('sigma')
#plt.title(r'$\mathrm{pred - true / true:}\ \mu=%.3f,\ \sigma=%.3f$' %(mu, sigma))
plt.title('Resolution: stochastic=%.3f,  constant=%.3f'%(popt[0], popt[1]))
print(popt,'\n',pcov)
plt.grid(True)

plt.show()

# PION PLOTS

In [ ]:
del pred,true,valloader
from tqdm import tqdm_notebook as tqdm
#mdl.zero_grad()
torch.cuda.empty_cache()
#model.eval()
pred = []
true = []
predc = []
misp = []
mist = []
#testloader2 = torch_geometric.data.DataLoader(data_list[totalev-3000:totalev], batch_size=1)
#testloader2 = torch_geometric.data.DataLoader(data_list, batch_size=50)
#valloader = torch_geometric.data.DataLoader(data_list_pi[totalevpi-14000:totalevpi], batch_size=30)
valloader = torch_geometric.data.DataLoader(data_list_pi, batch_size=60)



for data in tqdm(valloader):
    data = data.to(device)        
    result = mdl(data)
    #print(data.y)
    #print(result.i http://127.0.0.1:8005/?token=dcee9ad154badcbcaf8c1643ad620820eec6de751c69d92dtem().detach())
    #pred.append(result.item())
    #true.append(data.y.item())
    for i,j in zip(result,data.z.detach()):
        pred.append(i.detach().cpu()[2])
        predc.append(i.detach().cpu()[0:2].argmax())
        if (i.detach().cpu()[0:2].argmax() == 0):
            misp.append(i.detach().cpu()[2])
            mist.append(j.detach().cpu()[1])
        #pred.append(i.item())
#    print (pred) 
    #for i in data.z.detach():
        true.append(j.detach().cpu()[1])

#print()

In [ ]:
print(len(pred))
print(len(true))
print(np.unique(np.array(predc),return_counts=True))
print("pi acc, miss:",np.unique(np.array(predc),return_counts=True)[1][1]/len(predc),np.unique(np.array(predc),return_counts=True)[1][0]/len(predc))
#plt.hist(np.exp(mist),bins=50)
plt.scatter(np.exp(mist),np.exp(misp),alpha=0.3)
plt.xlabel("true")
plt.ylabel("pred")
plt.show()

In [ ]:
def plotpred(axs,pred,true,e1,e2):
#    preda = np.array(pred)
#    truea = np.array(true)
    fracarr = (pred - true)/true
#    if e2<250 : 
#        bin_heights, bin_borders, _ = plt.hist(fracarr,range=[-2,2], bins=20, label='histogram')
#    else:
    bin_heights, bin_borders, _ = axs.hist(fracarr,range=[-1.0,1.0], bins=100, label='histogram')

    bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2
    popt, pcov = curve_fit(gaussian, bin_centers, bin_heights, p0=[0., 1000., 0.1],bounds = ([-np.inf,0,0],[np.inf,np.inf,np.inf]))
    perr = np.sqrt(np.diag(pcov))
    #print(popt,perr)

    x_interval_for_fit = np.linspace(bin_borders[0], bin_borders[-1], 100)
    axs.plot(x_interval_for_fit, gaussian(x_interval_for_fit, *popt), label='fit')
    axs.legend()


    axs.set_xlabel('pred - true / true')
    axs.set_ylabel('counts')
    #plt.title(r'$\mathrm{pred - true / true:}\ \mu=%.3f,\ \sigma=%.3f$' %(mu, sigma))
    axs.set_title(r'$\mathrm{pred - true / true:}\ \mu=%.3f,\ \sigma=%.3f,\ E=$%i to %i' %(popt[0], popt[2],e1,e2))
    axs.grid(True)
    
#    plt.show()
    return [popt[0], popt[2],perr[0], perr[2]]

In [ ]:
#bins =  np.linspace(0,1000,21)
#bins =  np.linspace(0,250,6)
bins =  np.linspace(0,200,11)
bins2 = np.linspace(250,1000,16)
bins = np.append(bins,bins2)
print(bins)

pred = np.array(pred)
true = np.array(true)

print('before scaling pred',pred[0])
print('before scaling true',true[0])

pred  = np.exp(pred)
true = np.exp(true)
print('after scaling pred',pred[0])
print('after scaling true',true[0])




seoe = []
seoer = []
me = []
mer = []

fig, axs = plt.subplots(6,5, figsize=(40, 40), facecolor='w', edgecolor='k')
axs = axs.ravel()

for i in tqdm(range (bins.size - 1)):
    preda = pred[(true >bins[i]) & (true <bins[i+1]) ]
    truea = true[(true >bins[i]) & (true <bins[i+1]) ]
    vals = plotpred(axs[i],preda,truea,bins[i],bins[i+1])
    #print (vals)
    #seoe.append(vals[1]/bins[i+1])
    seoe.append(vals[1])
    seoer.append(vals[3])
    me.append(vals[0])
    mer.append(vals[2])
    #print (bins[i],bins[i+1])
seoear = np.array(seoe)
seoerar = np.array(seoer)
mear = np.array(me)
merar = np.array(mer)
#print(seoear,'\n',seoerar)
fig.savefig('outputs.pdf')

In [ ]:
plotbinner = lambda t: t+10 if (t<200) else t+25
vfunc = np.vectorize(plotbinner)
newbin = vfunc(bins[:bins.size-1])
#plt.plot(bins[:5]+50,(seoear)*100,yerr=seoerar)
plt.errorbar(newbin,(seoear), yerr=seoerar, fmt='o', ecolor='orangered',color='steelblue', capsize=2,ls='--');
plt.xlabel('Energy')
plt.ylabel('sigma')
plt.grid(True)

In [ ]:
plt.errorbar(newbin,(mear), yerr=merar, fmt='o', ecolor='orangered',color='steelblue', capsize=2,ls='--');
plt.xlabel('Energy')
plt.ylabel('mean')
plt.grid(True)

In [ ]:
#torch.load(i)
def reso(x,stoch,C):
#    return np.sqrt(stoch**2/x + e_dep**2/x**2 + C**2 )
    return np.sqrt(stoch**2/x  + C**2 )
 



popt, pcov = curve_fit(f=reso,xdata=newbin, ydata=seoear, sigma=seoerar, p0=[1.,1.],bounds = ([0,0],[np.inf,np.inf]))
perr = np.sqrt(np.diag(pcov))
print(popt,perr)

x_interval_for_fit = np.linspace(5,1000, 100)
plt.plot(x_interval_for_fit, reso(x_interval_for_fit, *popt), label='fit')
plt.errorbar(newbin,(seoear), yerr=seoerar, fmt='o', ecolor='orangered',color='steelblue', capsize=2,ls='--');
plt.legend()


plt.xlabel('Energy')
plt.ylabel('sigma')
#plt.title(r'$\mathrm{pred - true / true:}\ \mu=%.3f,\ \sigma=%.3f$' %(mu, sigma))
plt.title('Resolution: stochastic=%.3f,  constant=%.3f'%(popt[0], popt[1]))
print(popt,'\n',pcov)
plt.grid(True)

plt.show()